In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms

from torch.utils.data import Dataset, DataLoader
from DL_Studing.datasets.path_dataset import PathDataset


In [3]:
img = torch.randn(1, 7, 3, 3, 3)

In [4]:
img.shape

torch.Size([1, 7, 3, 3, 3])

In [14]:
[range(50 - 0 - 1)]

[range(0, 49)]

In [26]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
a[-1:len(a)]

[19]

In [27]:
a = torch.randn(16,1, 2)

In [31]:
a.squeeze(1).shape

torch.Size([16, 2])

In [3]:
a = [(0,1),(2,2),(3,3),(4,4),(5,5),(6,6),(7,7)]
a[-1][1]

7